In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
26958,2024-12-08,Noruega Blno,12:00,Oppsal,Froya,8.40,28.00,167.5,1.83,1.83,29.5,2.01,1.68,Gpz6dxcl,0.119048,0.035714,0.546448,0.546448,-0.845238,525.870,291.564726,0.554443,0.6,1.341641,2.236068,475.00,8.326,4.345904,0.521968,-215.0,213.936,158.968191,0.743064,1.2,1.643168,1.369306,161.19,2.922,2.046771,0.700469,18.0,50,81,9.50,1.99,445.096,184.736,0.761500,0.000000,0.126474,-2.83,-0.566,-13.074205,0.332473,0.2,-0.132473,2.07,0.414,65.217391,0.578487,0.4,-0.178487
26959,2024-12-08,Países Baixos Wbl Feminina,12:00,Leiden F,Den Helder F,4.31,1.16,137.5,1.88,1.78,10.5,1.92,1.75,rmfivCcn,0.232019,0.862069,0.531915,0.561798,0.094088,204.264,177.967274,0.871261,0.6,1.341641,2.236068,88.92,3.686,3.456708,0.937794,-73.0,155.316,91.148869,0.586861,0.6,1.341641,2.236068,234.72,2.476,1.549332,0.625740,-25.0,52,72,1.71,3.26,0.000,0.000,0.814401,0.038640,0.065509,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
26960,2024-12-08,Países Baixos Wbl Feminina,15:00,Limburg F,Zwolle F,1.04,7.85,130.5,1.83,1.83,-17.5,2.04,1.66,4MOtK8CU,0.961538,0.127389,0.546448,0.546448,0.088927,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,75,59,1.97,6.98,0.000,0.000,1.083329,0.000000,0.145244,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
26961,2024-12-08,Polônia Liga De Basquete,11:30,Ostrow Wielkopolski,GTK Gliwice,1.26,3.35,171.5,1.83,1.83,-7.5,1.92,1.75,tnCCdGt3,0.793651,0.298507,0.546448,0.546448,0.092158,237.020,79.687416,0.336205,0.6,1.341641,2.236068,139.20,2.960,1.018627,0.344131,-20.0,215.608,92.890424,0.430830,1.2,1.643168,1.369306,240.24,2.638,1.337356,0.506958,-65.0,80,84,1.74,2.86,146.072,294.670,0.641151,0.000000,0.065509,-1.93,-0.386,-0.673575,0.681999,0.6,-0.081999,0.81,0.162,14.506173,0.318083,0.4,0.081917
26962,2024-12-08,Macedônia Do Norte Prva Liga,14:00,Tikvesh,Vardar,1.23,3.88,159.5,1.83,1.83,-9.5,1.83,1.83,KdufQ5Ku,0.813008,0.257732,0.546448,0.546448,0.070740,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,499.694,461.498066,0.923561,0.6,1.341641,2.236068,468.47,6.464,5.854206,0.905663,-29.0,82,79,1.35,5.93,0.000,505.666,0.733398,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,-0.92,-0.184,-15.652174,0.286588,0.2,-0.086588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27174,2024-12-08,Eua Ncaa,18:00,Murray State,SE Missouri State,1.07,7.28,142.5,1.85,1.85,-17.5,2.00,1.72,21KkT9aD,0.934579,0.137363,0.540541,0.540541,0.071942,162.218,161.291766,0.994290,1.8,1.643168,0.912871,68.04,2.294,2.414463,1.052512,16.0,372.360,241.580125,0.648781,1.2,1.643168,1.369306,552.00,5.898,4.318011,0.732114,-29.0,63,60,1.08,9.20,85.268,449.868,1.051768,0.000000,0.106446,-1.42,-0.284,-0.246479,0.754485,0.6,-0.154485,0.29,0.058,108.275862,0.235556,0.2,-0.035556
27175,2024-12-08,Eua Ncaa,18:00,South Alabama,Jacksonville St.,1.69,2.07,133.5,1.85,1.85,-3.5,1.96,1.75,0OdqoWqC,0.591716,0.483092,0.540541,0.540541,0.074808,208.732,261.111912,1.250943,1.2,1.6

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
26986,12:00,República Tcheca Zbl Feminina,USK Prague F,Slovanka F,1.65,Back Home
27030,14:30,Macedônia Do Norte Prva Liga,Pelister,Cair 2030,2.24,Back Home
27049,06:00,Taiwan Tpbl,Taiwan Beer Leopards,Taipei Taishin Mars,1.57,Back Home
27064,20:30,Eua Nba,Orlando Magic,Phoenix Suns,1.89,Back Home
27070,13:30,Áustria Liga Zweite,Vienna Timberwolves,Mattersburg,1.64,Back Home
27072,08:00,Bulgária Nbl,Academic Plovdiv,Minyor,1.62,Back Home
27097,12:00,Suécia Superettan,Aros Basket,IK Eos,1.58,Back Home
27098,08:30,Espanha Acb,Bilbao,Baskonia,2.06,Back Home
27140,15:00,França Lnb,Paris,Monaco,1.68,Back Home
27153,08:00,Espanha Liga Feminina,Joventut F,Gernika Bizkaia F,1.58,Back Home
